<a href="https://colab.research.google.com/github/Mchoon84/AIFFEL_guest_cr/blob/main/Exploration/Ex06/Ex06_%EB%85%B8%EB%93%9C11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 준비하기
✅ Diffusers를 설치

In [ ]:
!pip install --upgrade -qq git+https://github.com/huggingface/diffusers.git transformers accelerate
# !pip install -U triton


In [ ]:
import diffusers
print(diffusers.__version__)


# Text-to-Image Generation
✅ 파이프라인 불러오기

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler

device = "cuda"

# 파이프라인 불러오기
repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16)

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)

✅ 프롬프트 입력  

Positive Prompt:  

a self-portrait of Vincent van Gogh wearing a detailed futuristic astronaut suit, surreal cosmic background with swirling nebulae and stars, post-impressionist brush stroke texture, expressive and introspective face, vibrant Van Gogh color palette, dramatic galactic lighting, digital oil painting, high detail, 8K resolution, cinematic depth, fusion of classic and sci-fi, textured canvas style, masterpiece, trending on ArtStation

In [ ]:
prompt = str(input('prompt: '))  # 프롬프트를 영어로 입력해보세요

In [ ]:
# 입력한 프롬프트를 사용하여 이미지 생성
image = pipe(prompt, num_inference_steps=25).images[0]

# 이미지 출력
image

✅ 하이퍼파라미터를 변화

- height, width: 생성될 이미지의 가로와 세로 픽셀 크기를 조절할 수 있습니다. 8의 배수로 설정해 주세요.
- num_inference_steps: denoising 스텝 수로, 값이 커질수록 고해상도 이미지가 출력되지만 출력되는 시간이 오래 걸립니다. default 값은 50입니다.
- guidance_scale: 얼마나 주어진 프롬프트에 근접한 이미지를 생성할지를 설정하는 하이퍼파라미터로, 값이 커질수록 문자열에 근접한 이미지가 생성되지만 이미지 품질이 떨어질 수 있습니다. default는 7.5입니다.





### 실험 1 : 이미지 사이즈 변화
- 크기가 작으면 결과가 더 잘나올거 같음
- 프롬프트 동일하게 크기만 변경

In [14]:
# ===== 사용자 지정 하이퍼파라미터 =====
HEIGHT  = 320   # 8의 배수
WIDTH   = 320   # 8의 배수
STEPS   = 50    # num_inference_steps (기본 50)
GUIDANCE = 7.5  # guidance_scale (기본 7.5)
# ====================================


In [ ]:
prompt = str(input('prompt: '))  # 프롬프트를 영어로 입력해보세요

In [ ]:
image = pipe(
    prompt,
    height=HEIGHT,
    width=WIDTH,
    num_inference_steps=STEPS,
    guidance_scale=GUIDANCE
).images[0]

# 이미지 출력
image

### 실험 2 : 스텝수 변경
- 50 에서 100 으로 변경
- 고해상도 이미지를 만들어보자


In [28]:
# ===== 사용자 지정 하이퍼파라미터 =====
HEIGHT  = 320   # 8의 배수
WIDTH   = 320   # 8의 배수
STEPS   = 100    # num_inference_steps (기본 50)
GUIDANCE = 7.5  # guidance_scale (기본 7.5)
# ====================================


In [ ]:
prompt = str(input('prompt: '))  # 프롬프트를 영어로 입력해보세요

In [ ]:
image = pipe(
    prompt,
    height=HEIGHT,
    width=WIDTH,
    num_inference_steps=STEPS,
    guidance_scale=GUIDANCE
).images[0]

# 이미지 출력
image

✅ 해삳도를 높이니, 그림이 사진처럼 되어버림   
✅ 반 고흐 사진에 그림 마스크를 합성한 느낌

### 실험 3 : guidance_scale 하향 조정
- 2 정도로 조정
- 프롬프트와 상관없는 이미지 생성 목적

In [35]:
# ===== 사용자 지정 하이퍼파라미터 =====
HEIGHT  = 320   # 8의 배수
WIDTH   = 320   # 8의 배수
STEPS   = 50    # num_inference_steps (기본 50)
GUIDANCE = 2  # guidance_scale (기본 7.5)
# ====================================


In [ ]:
prompt = str(input('prompt: '))  # 프롬프트를 영어로 입력해보세요

In [ ]:
image = pipe(
    prompt,
    height=HEIGHT,
    width=WIDTH,
    num_inference_steps=STEPS,
    guidance_scale=GUIDANCE
).images[0]

# 이미지 출력
image

✅ 전혀 상관없는 품질이 떨어지는 그림이 나옴

# 여러 개의 이미지 생성하기

파이썬 라이브러리인 pillow를 사용

In [45]:
# 파이썬 이미지 처리 라이브러리 pillow 불러오기
from PIL import Image

# 틀 만들기
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows * cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols * w, rows * h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box = (i%cols * w, i // cols * h))
    return grid

In [49]:
# ===== 사용자 지정 하이퍼파라미터 =====
HEIGHT  = 320   # 8의 배수
WIDTH   = 320   # 8의 배수
STEPS   = 50    # num_inference_steps (기본 50)
GUIDANCE = 7.5  # guidance_scale (기본 7.5)
# ====================================

In [ ]:
# 이미지의 개수
num_images = 6

# 프롬프트 입력
prompt = "a self-portrait of Vincent van Gogh wearing a detailed futuristic astronaut helmet"
prompts = [prompt] * num_images

# 이미지 생성
images = [pipe(
    prompt,
    height=HEIGHT,
    width=WIDTH,
    num_inference_steps=STEPS,
    guidance_scale=GUIDANCE
).images[0] for _ in range(num_images)]

# 이미지 출력
grid = image_grid(images, rows= 3, cols= 2)
grid

✅ 처음에는 쓸만한게 없었는데 계속 돌리다 보니 괜찮은 이미지가 생성  
✅ 같은 프롬프트를 써서 여러장의 이미지를 출력하는 것이 하나의 이미지를 생성 하는 것 보다 선택의 폭이 넓어짐